In [19]:
import warnings
warnings.filterwarnings('ignore')

In [20]:
def get_basic_position(style):
    
    print(style)

    splitted = style.split('(')
    width = float(splitted[2].split('%')[0])
    height = float(splitted[1].split('%')[0])
    
    group = ""
    
    if width < 10:
        group = "GK"
    elif width < 20:
        group = "DF"
    elif width < 40:
        group = "MF"
    else:
        group = "FW"
  
    return group

In [37]:
def get_home_lineup(comp, year, home_team, away_team):
    
    from scores_and_fixtures import season_results_and_fixtures
    from internal_packages import check_name
    from internal_packages.get_position import get_basic_position, get_detailed_position
    from datetime import datetime
    import pandas as pd
    import requests
    from bs4 import BeautifulSoup

    results_df = season_results_and_fixtures(comp, year)
    home_team = check_name.get_team_name(home_team)
    away_team = check_name.get_team_name(away_team)
    
    for i, row in results_df.iterrows():
        if row['home_team'] == home_team and row['away_team'] == away_team:
            url = f"https://fbref.com/{row['match_url']}"
            resp = requests.get(url) 
            resp.encoding = 'utf-8'
            soup = BeautifulSoup(resp.text.replace('<!--', '').replace('--!>', ''), 'html.parser')
            
            div_lineup_home = soup.find('div', {'class':'lineup'}, {'id':'a'})
            numbers, player_names, positions = [], [], []
            
            players = div_lineup_home.find_all('td')
                
            for i, value in enumerate(players):
                if i % 2 == 0:
                    numbers.append(value.get_text())
                elif i % 2 == 1: 
                    player_names.append(value.get_text())
                    
            df = pd.DataFrame(list(zip(numbers, player_names)), columns = ['squad_number', 'player'])
            
            positions = []
            
            for i, row in df.iterrows():
                try:
                    player_div = soup.find("div", {"title":{row['player']}})
                    style = player_div["style"]
                    get_position = get_basic_position(style)
                    df.at[i, 'position'] = get_position
                    positions.append(get_position)
                except: pass
                
            keepers = positions.count("GK")
            defenders = positions.count("DF")
            midfielders = positions.count("MF")
            forwards = positions.count("FW")
                
            for i, row in df.iterrows():
                try:
                    player_div = soup.find("div", {"title":{row['player']}})
                    style = player_div["style"]
                    get_det_position = get_detailed_position(style, keepers, defenders, midfielders, forwards)
                    df.at[i, 'det_pos'] = get_det_position
                except: pass

    return df

In [38]:
get_home_lineup("ger", 2023, "bayern", "bvb")

5.0
50.0
15.0
66.6666666666667
15.0
83.3333333333333
25.0
33.3333333333333
45.0
50.0
40.0
83.3333333333333
15.0
33.3333333333333
15.0
16.6666666666667
35.0
50.0
25.0
66.6666666666667
40.0
16.6666666666667
5.0
50.0
15.0
66.6666666666667
15.0
83.3333333333333
25.0
33.3333333333333
45.0
50.0
40.0
83.3333333333333
15.0
33.3333333333333
15.0
16.6666666666667
35.0
50.0
25.0
66.6666666666667
40.0
16.6666666666667


,squad_number,player,position,det_pos
0,26,Sven Ulreich,GK,GK
1,4,Matthijs de Ligt,DF,CB
2,6,Joshua Kimmich,DF,WB
3,8,Leon Goretzka,MF,DM
4,9,Harry Kane,FW,ST
5,10,Leroy Sané,FW,WF
6,15,Eric Dier,DF,CB
7,19,Alphonso Davies,DF,WB
8,25,Thomas Müller,MF,AM
9,27,Konrad Laimer,MF,DM
